In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import re

In [2]:
url = "https://www.pro-football-reference.com/years/{year}/scrimmage.htm".format(year="{year}")

In [3]:
res = requests.get(url.format(year='2020'), verify=True)

In [4]:
soup = BeautifulSoup(res.text, "html.parser")

In [5]:
parse = soup.findAll('table', {
    'id':'receiving_and_rushing'
})

In [15]:
data_stats = soup.findAll('td', {
    'data-stat': 'player'
})

In [19]:
len(data_stats)

580

In [20]:
data_stats_pos = soup.findAll('td', {
    'data-stat': 'pos'
})

In [21]:
len(data_stats_pos)

580

In [23]:
data_stats_pos[0].text

'RB'

In [8]:
baseURL = 'https://www.pro-football-reference.com'

In [9]:
# https://www.pro-football-reference.com/players/H/HenrDe00/gamelog

In [27]:
players = []
z = 0

for row in data_stats:
    pname = row.text.replace(' *', '').replace('*', '').replace('+', '')
    ppos = data_stats_pos[z].text
    print(pname)
    phref = row.findAll(href=True)[0]['href']
    fullURL = baseURL + phref[:-4] + '/gamelog'
    r = requests.get(fullURL, verify=True)
    zoup = BeautifulSoup(r.text, 'html.parser')
    parze = zoup.findAll('table', {
    'id': 'stats'
    })
    
    player_data = {}
    p_data = []
    i = -1

    for a in parze[0].findAll('td'):
        data_field = a['data-stat']
        data_stat = a.text #.replace(' *', '').replace('*', '').replace('+', '')

        if data_field == 'year_id':
            i += 1
            p_data.append(player_data)
            player_data = {}

        player_data.update({data_field: data_stat})
        
    p_data = p_data[1:]
    temp_df = pd.DataFrame(p_data)
    temp_df.insert(0, 'player_name', pname)
    temp_df.insert(1, 'player_pos', ppos)
    temp_df.to_excel(f'player_files/{pname}.xlsx', index=False)
    z += 1

Derrick Henry
Dalvin Cook
Alvin Kamara
Stefon Diggs
David Montgomery
Jonathan Taylor
Aaron Jones
Travis Kelce
James Robinson
DeAndre Hopkins
Justin Jefferson
Tyreek Hill
Calvin Ridley
Davante Adams
Ezekiel Elliott
Josh Jacobs
D.K. Metcalf
Allen Robinson
Nick Chubb
D.J. Moore
Darren Waller
Brandin Cooks
Terry McLaurin 
Kareem Hunt
Melvin Gordon
Ronald Jones II
Amari Cooper
Robby Anderson
Clyde Edwards-Helaire 
Kenyan Drake
Robert Woods
A.J. Brown
Miles Sanders 
Tyler Lockett
Curtis Samuel 
Antonio Gibson 
CeeDee Lamb 
Mike Davis 
Cooper Kupp
Mike Evans
Lamar Jackson
David Johnson
Keenan Allen
Corey Davis 
Marvin Jones 
Myles Gaskin
Chris Carson
Cole Beasley
Devin Singletary
J.D. McKissic
Adam Thielen
Diontae Johnson 
James Conner
Tee Higgins 
Austin Ekeler
J.K. Dobbins 
Nelson Agholor
Tyler Boyd
Chase Claypool 
Will Fuller
D'Andre Swift 
Nyheim Hines
Jerry Jeudy 
Gus Edwards 
Chase Edmonds
Jarvis Landry 
Michael Gallup
Todd Gurley
Chris Godwin
Latavius Murray
JuJu Smith-Schuster
Brandon

Jeff Heath
Kendall Lamm


In [28]:
from glob import glob as gb

In [29]:
player_files = gb(r'player_files/*')

In [220]:
data = []

for pf in player_files:
    df = pd.read_excel(pf)
    data.append(df)
    
player_df = pd.concat(data)

player_df

,player_name,player_pos,year_id,game_date,game_num,week_num,age,team,game_location,opp,...,punt_yds,punt_yds_per_punt,punt_blocked,xpm,xpa,xp_perc,fgm,fga,fg_perc,safety_md
0,A.J. Brown,WR,2019.0,2019-09-08,1.0,1.0,22.070,TEN,@,CLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A.J. Brown,WR,2019.0,2019-09-15,2.0,2.0,22.077,TEN,NaN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A.J. Brown,WR,2019.0,2019-09-19,3.0,3.0,22.081,TEN,@,JAX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A.J. Brown,WR,2019.0,2019-09-29,4.0,4.0,22.091,TEN,@,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A.J. Brown,WR,2019.0,2019-10-06,5.0,5.0,22.098,TEN,NaN,BUF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,Zeke Turner,NaN,2020.0,2020-12-06,12.0,13.0,24.180,ARI,NaN,LAR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,Zeke Turner,NaN,2020.0,2020-12-13,13.0,14.0,24.187,ARI,@,NYG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,Zeke Turner,NaN,2020.0,2020-12-20,14.0,15.0,24.194,ARI,NaN,PHI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,Zeke Turner,NaN,2020.0,2020-12-26,15.0,16.0,24.200,ARI,NaN,SFO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
player_df.columns

Index(['player_name', 'player_pos', 'year_id', 'game_date', 'game_num',
       'week_num', 'age', 'team', 'game_location', 'opp', 'game_result', 'gs',
       'targets', 'rec', 'rec_yds', 'rec_yds_per_rec', 'rec_td', 'catch_pct',
       'rec_yds_per_tgt', 'rush_att', 'rush_yds', 'rush_yds_per_att',
       'rush_td', 'kick_ret', 'kick_ret_yds', 'kick_ret_yds_per_ret',
       'kick_ret_td', 'all_td', 'scoring', 'fumbles', 'fumbles_lost',
       'fumbles_forced', 'fumbles_rec', 'fumbles_rec_yds', 'fumbles_rec_td',
       'offense', 'off_pct', 'defense', 'def_pct', 'special_teams', 'st_pct',
       'reason', 'def_int', 'def_int_yds', 'def_int_td', 'pass_defended',
       'pass_cmp', 'pass_att', 'pass_cmp_perc', 'pass_yds', 'pass_td',
       'pass_int', 'pass_rating', 'pass_sacked', 'pass_sacked_yds',
       'pass_yds_per_att', 'pass_adj_yds_per_att', 'two_pt_md', 'punt_ret',
       'punt_ret_yds', 'punt_ret_yds_per_ret', 'punt_ret_td', 'sacks',
       'tackles_solo', 'tackles_assists', 'tac

In [221]:
parse_player_df = player_df.columns[:23]

In [222]:
player_files[1]

'player_files\\A.J. Green.xlsx'

In [214]:
# start compiling into single action

In [232]:
team_nums = pd.read_excel(r'team_names.xlsx')
train_data = []

for player in player_df['player_name'].unique():
    player_ = player_df[player_df['player_name'] == player]
    player_ = player_[list(parse_player_df) + ['reason']]
    player_['age'] = player_['age'].ffill().bfill()
    player_ = player_.fillna(0).replace('@', 1).replace('*', 1).replace('%', '', regex=True)
    player_['catch_pct'] = player_['catch_pct'].astype(float)
    player_['reason'] = player_['reason'].replace(r"[^0-9]+", 1, regex=True)
    player_['opp'] = player_['opp'].map(dict(zip(team_nums['Abbr'], team_nums['Team_Number'])))
    player_['team'] = player_['team'].map(dict(zip(team_nums['Abbr'], team_nums['Team_Number'])))
    cols = player_.columns[4:]

    career_data = []

    for yr in player_['year_id'].unique():
        df = player_[player_['year_id'] <= yr].groupby(['game_num']).mean().reset_index()
        df.insert(0, 'yr', yr)
        career_data.append(df)

    career_df_ = pd.concat(career_data).reset_index().drop(['index'], axis=1)
    career_df_.columns = career_df_.columns.map(lambda x: 'career_' + str(x))

    player_model = player_.join(career_df_)

    train_data.append(player_model)
        
train_df = pd.concat(train_data)
train_df.insert(0, 'fp', train_df['rec'] + train_df['rec_yds'] * 0.1 + train_df['rush_yds'] * 0.1)
train_df.to_csv('train.csv', index=False)

In [215]:
####

In [173]:
aj = pd.read_excel(player_files[1])

In [174]:
aj = aj[list(parse_player_df) + ['reason']]

In [175]:
aj['age'] = aj['age'].ffill().bfill()
aj = aj.fillna(0).replace('@', 1).replace('*', 1).replace('%', '', regex=True)
aj['catch_pct'] = aj['catch_pct'].astype(float)
aj['reason'] = aj['reason'].replace(r"[^0-9]+", 1, regex=True)

In [176]:
team_nums = pd.read_excel(r'team_names.xlsx')

In [178]:
aj['opp'] = aj['opp'].map(dict(zip(team_nums['Abbr'], team_nums['Team_Number'])))
aj['team'] = aj['team'].map(dict(zip(team_nums['Abbr'], team_nums['Team_Number'])))

In [181]:
aj['year_id'].unique()

array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020],
      dtype=int64)

In [182]:
aj_copy = aj.copy()

In [183]:
cols = aj_copy.columns[4:]

Index(['game_num', 'week_num', 'age', 'team', 'game_location', 'opp',
       'game_result', 'gs', 'targets', 'rec', 'rec_yds', 'rec_yds_per_rec',
       'rec_td', 'catch_pct', 'rec_yds_per_tgt', 'rush_att', 'rush_yds',
       'rush_yds_per_att', 'rush_td', 'reason'],
      dtype='object')

In [184]:
aj_2019 = aj_copy[aj_copy['year_id'] <= 2019][cols]

In [203]:
career_data = []

for yr in aj_copy['year_id'].unique():
    if yr == 2020:
        break
    df = aj_copy[aj_copy['year_id'] <= yr].groupby(['game_num']).mean().reset_index()
    df.insert(0, 'yr', yr)
    career_data.append(df)
    
career_df = pd.concat(career_data).reset_index().drop(['index'], axis=1)

In [204]:
career_df.sort_values(['year_id','game_num'])

,yr,game_num,year_id,week_num,age,team,game_location,opp,gs,targets,...,rec_yds,rec_yds_per_rec,rec_td,catch_pct,rec_yds_per_tgt,rush_att,rush_yds,rush_yds_per_att,rush_td,reason
0,2011,1,2011.0,1.0,23.042000,7.0,1.000000,8.000000,1.000000,4.000000,...,41.000000,41.000000,1.000000,25.000000,10.250000,0.000000,0.000000,0.000000,0.0,0.000000
1,2011,2,2011.0,2.0,23.049000,7.0,1.000000,10.000000,1.000000,14.000000,...,124.000000,12.400000,1.000000,71.400000,8.860000,0.000000,0.000000,0.000000,0.0,0.000000
2,2011,3,2011.0,3.0,23.056000,7.0,0.000000,29.000000,1.000000,5.000000,...,29.000000,7.250000,0.000000,80.000000,5.800000,0.000000,0.000000,0.000000,0.0,0.000000
3,2011,4,2011.0,4.0,23.063000,7.0,0.000000,4.000000,1.000000,10.000000,...,118.000000,29.500000,0.000000,40.000000,11.800000,1.000000,6.000000,6.000000,0.0,0.000000
4,2011,5,2011.0,5.0,23.070000,7.0,1.000000,15.000000,1.000000,8.000000,...,90.000000,18.000000,1.000000,62.500000,11.250000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2019,12,2015.0,13.0,27.012111,7.0,0.555556,20.777778,0.777778,6.888889,...,58.222222,10.932222,0.666667,52.466667,7.397778,0.111111,1.666667,1.666667,0.0,0.222222
140,2019,13,2015.0,14.0,27.016667,7.0,0.333333,14.333333,0.666667,6.555556,...,66.111111,10.403333,0.333333,40.622222,6.428889,0.000000,0.000000,0.000000,0.0,0.333333
141,2019,14,2015.0,15.0,27.021000,7.0,0.666667,22.000000,0.666667,6.111111,...,42.333333,11.200000,0.111111,33.944444,5.325556,0.000000,0.000000,0.000000,0.0,0.333333
142,2019,15,2015.0,16.0,27.026111,7.0,0.555556,13.444444,0.666667,6.777778,...,41.777778,6.984444,0.333333,28.277778,3.564444,0.222222,2.777778,1.388889,0.0,0.333333


In [205]:
career_df.columns = career_df.columns.map(lambda x: 'career_' + str(x))

In [206]:
aj_model = aj[aj['year_id'] <= 2019].join(career_df)

In [207]:
aj_model

,player_name,player_pos,year_id,game_date,game_num,week_num,age,team,game_location,opp,...,career_rec_yds,career_rec_yds_per_rec,career_rec_td,career_catch_pct,career_rec_yds_per_tgt,career_rush_att,career_rush_yds,career_rush_yds_per_att,career_rush_td,career_reason
0,A.J. Green,WR,2011,2011-09-11,1,1,23.042,7,1,8,...,41.000000,41.000000,1.000000,25.000000,10.250000,0.000000,0.000000,0.000000,0.0,0.000000
1,A.J. Green,WR,2011,2011-09-18,2,2,23.049,7,1,10,...,124.000000,12.400000,1.000000,71.400000,8.860000,0.000000,0.000000,0.000000,0.0,0.000000
2,A.J. Green,WR,2011,2011-09-25,3,3,23.056,7,0,29,...,29.000000,7.250000,0.000000,80.000000,5.800000,0.000000,0.000000,0.000000,0.0,0.000000
3,A.J. Green,WR,2011,2011-10-02,4,4,23.063,7,0,4,...,118.000000,29.500000,0.000000,40.000000,11.800000,1.000000,6.000000,6.000000,0.0,0.000000
4,A.J. Green,WR,2011,2011-10-09,5,5,23.070,7,1,15,...,90.000000,18.000000,1.000000,62.500000,11.250000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,A.J. Green,WR,2019,2019-12-01,12,13,30.124,7,0,25,...,58.222222,10.932222,0.666667,52.466667,7.397778,0.111111,1.666667,1.666667,0.0,0.222222
140,A.J. Green,WR,2019,2019-12-08,13,14,30.124,7,1,8,...,66.111111,10.403333,0.333333,40.622222,6.428889,0.000000,0.000000,0.000000,0.0,0.333333
141,A.J. Green,WR,2019,2019-12-15,14,15,30.124,7,0,23,...,42.333333,11.200000,0.111111,33.944444,5.325556,0.000000,0.000000,0.000000,0.0,0.333333
142,A.J. Green,WR,2019,2019-12-22,15,16,30.124,7,1,20,...,41.777778,6.984444,0.333333,28.277778,3.564444,0.222222,2.777778,1.388889,0.0,0.333333


In [273]:
data = pd.read_csv('model.csv')

In [274]:
data = data.drop(['game_result', 'career_reason'], axis=1).fillna(0)

In [275]:
data.head()

,fp,player_name,player_pos,year_id,game_date,game_num,week_num,age,team,game_location,...,career_rec_yds,career_rec_yds_per_rec,career_rec_td,career_catch_pct,career_rec_yds_per_tgt,career_rush_att,career_rush_yds,career_rush_yds_per_att,career_rush_td,career_player_pos
0,13.0,A.J. Brown,WR,2019,2019-09-08,1,1,22.070,31,1,...,100.0,33.33,0.0,75.0,25.00,0.0,0.0,0.0,0.0,0.0
1,5.5,A.J. Brown,WR,2019,2019-09-15,2,2,22.077,31,0,...,25.0,8.33,0.0,60.0,5.00,0.0,0.0,0.0,0.0,0.0
2,1.4,A.J. Brown,WR,2019,2019-09-19,3,3,22.081,31,1,...,4.0,4.00,0.0,20.0,0.80,0.0,0.0,0.0,0.0,0.0
3,12.4,A.J. Brown,WR,2019,2019-09-29,4,4,22.091,31,1,...,94.0,31.33,2.0,100.0,31.33,0.0,0.0,0.0,0.0,0.0
4,4.7,A.J. Brown,WR,2019,2019-10-06,5,5,22.098,31,0,...,27.0,13.50,0.0,100.0,13.50,0.0,0.0,0.0,0.0,0.0


In [276]:
train = data[data['year_id'] <= 2019]
test = data[data['year_id'] == 2020]

In [289]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [290]:
Y_train = train['fp']
X_train = train[train.columns[5:]]

In [291]:
Y_test = test['fp']
X_test = test[test.columns[5:]]

In [292]:
X_train.shape, Y_train.shape

((28405, 40), (28405,))

In [293]:
X_test.shape, Y_test.shape

((8514, 40), (8514,))

In [305]:
model = linear_model.LinearRegression()

In [306]:
model.fit(X_train, Y_train)

LinearRegression()

In [307]:
Y_pred = model.predict(X_test)

In [308]:
r2 = model.score(X_train, Y_train)
n = X_train.shape[0]
p = X_train.shape[1]

adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)

In [309]:
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(Y_test, Y_pred))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(Y_test, Y_pred))
print('Adjusted Coefficient of determination (adj R^2): %.2f' % adjusted_r2)

Coefficients: [ 6.39817108e-15 -5.22845656e-15 -5.95186335e-16  3.97832741e-16
 -6.50960067e-16  1.06617731e-16  2.18796742e-15 -5.34849129e-15
  1.00000000e+00  1.00000000e-01  7.33137509e-16 -4.69050170e-15
  2.77772597e-16 -3.60537880e-16 -5.92382317e-15  1.00000000e-01
 -2.44636668e-16  1.11787822e-15  2.17752181e-15 -7.22803707e-16
 -1.70870262e-15 -7.64091481e-17  9.52363188e-16  1.19587500e-16
 -2.22007335e-16  2.72369373e-16  2.16623594e-16 -1.94725759e-15
 -8.61493494e-16  2.56271597e-15 -5.23263073e-17  6.16812780e-16
 -1.13031465e-16 -1.55501697e-16 -7.00506835e-17  2.75547441e-16
  1.68281730e-16  1.48896195e-15  2.54137815e-15  0.00000000e+00]
Intercept: 1.6155965454345278e-12
Mean squared error (MSE): 0.00
Coefficient of determination (R^2): 1.00
Adjusted Coefficient of determination (adj R^2): 1.00


In [310]:
data.columns

Index(['fp', 'player_name', 'player_pos', 'year_id', 'game_date', 'game_num',
       'week_num', 'age', 'team', 'game_location', 'opp', 'gs', 'targets',
       'rec', 'rec_yds', 'rec_yds_per_rec', 'rec_td', 'catch_pct',
       'rec_yds_per_tgt', 'rush_att', 'rush_yds', 'rush_yds_per_att',
       'rush_td', 'reason', 'career_yr', 'career_game_num', 'career_year_id',
       'career_week_num', 'career_age', 'career_team', 'career_game_location',
       'career_opp', 'career_gs', 'career_targets', 'career_rec',
       'career_rec_yds', 'career_rec_yds_per_rec', 'career_rec_td',
       'career_catch_pct', 'career_rec_yds_per_tgt', 'career_rush_att',
       'career_rush_yds', 'career_rush_yds_per_att', 'career_rush_td',
       'career_player_pos'],
      dtype='object')